# 特征值选择实验项目

在机器学习项目中，特征选择是一个非常重要的步骤，它可以帮助提高模型的性能、减少计算成本，并提高模型的解释性。特征选择可以通过不同的方法进行，包括过滤法（Filter Methods）、包裹法（Wrapper Methods）和嵌入法（Embedded Methods）。本Notebook将介绍如何使用Python中的几种常见库（如scikit-learn、pandas和numpy）来进行特征选择。

## 1. 准备环境

首先，确保你已经安装了必要的Python库。如果没有安装，可以使用pip来安装：

```bash
pip install numpy pandas scikit-learn matplotlib
```

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## 2. 加载数据

我们将使用scikit-learn库中的数据集，例如鸢尾花（Iris）数据集作为示例。

In [ ]:
# 加载数据
data = load_iris()
X = data.data
y = data.target

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 设置 random_state 以保证结果可复现

## 3. 过滤法（Filter Methods）

过滤法通过计算每个特征与目标变量之间的统计相关性来选择特征。例如，可以使用皮尔逊相关系数或卡方检验。

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# 使用卡方检验选择最佳特征
selector = SelectKBest(chi2, k=2)  # 选择前两个最佳特征
X_train_new = selector.fit_transform(X_train, y_train)
X_test_new = selector.transform(X_test)

## 4. 包裹法（Wrapper Methods）

包裹法通过训练一个模型来评估每个特征的重要性。例如，可以使用递归特征消除（RFE）或基于树的特征选择。

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

# 使用随机森林进行特征选择
model = RandomForestClassifier(n_estimators=100, random_state=42) # 设置 random_state 以保证结果可复现
rfe = RFE(model, n_features_to_select=2)
X_train_new = rfe.fit_transform(X_train, y_train)
X_test_new = rfe.transform(X_test)

## 5. 嵌入法（Embedded Methods）

嵌入法是将特征选择融入到模型训练过程中的方法，例如使用基于树的模型如随机森林或梯度提升树。这些模型在训练时会考虑特征的重要性。

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

# 使用梯度提升树进行特征选择
model = GradientBoostingClassifier(random_state=42) # 设置 random_state 以保证结果可复现
model.fit(X_train, y_train)
sfm = SelectFromModel(model, prefit=True)  # 使用模型选择特征
X_train_new = sfm.transform(X_train)
X_test_new = sfm.transform(X_test)

## 6. 评估模型性能

最后，使用选定的特征训练模型并评估其性能。

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC  # 以支持向量机为例进行分类任务

# 使用SVM进行分类并评估模型性能
clf = SVC()
clf.fit(X_train_new, y_train)
y_pred = clf.predict(X_test_new)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

以上步骤展示了如何在Python中使用不同的方法进行特征选择，并使用选定的特征训练一个机器学习模型。你可以根据具体的数据集和问题选择合适的方法和模型。